In [1]:
from PIL import Image

from tensorflow.keras.models import load_model
import numpy as np
from numpy import asarray
from numpy import expand_dims
from keras_facenet import FaceNet
import pickle
import cv2
import uuid
import os

In [2]:
#HaarCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
MyFaceNet = FaceNet()

In [3]:
myfile = open("data.pkl", "rb")
database = pickle.load(myfile)
myfile.close()

In [4]:
def onlychar(input_string):
    return ''.join(char for char in input_string if char.isalpha() or char.isspace())

In [5]:
cap = cv2.VideoCapture(0)

while(1):
    _, gbr1 = cap.read()
    
    wajah = HaarCascade.detectMultiScale(gbr1, 1.1, 4)
    
    if len(wajah) > 0:
        x1, y1, width, height = wajah[0] 
        x1, y1, x2, y2 = x1, y1, x1 + width, y1 + height  # Updated this line
        
        x1, y1 = abs(x1), abs(y1)
    
        gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
        gbr = Image.fromarray(gbr)                  
        gbr_array = asarray(gbr)
        
        face = gbr_array[y1:y2, x1:x2]                        
        
        face = Image.fromarray(face)                       
        face = face.resize((160, 160))
        face = asarray(face)
        face = expand_dims(face, axis=0)
        signature = MyFaceNet.embeddings(face)
        
        min_dist = 100
        identity = 'Unknown'  # Default to 'Unknown'
        
        for key, value in database.items():
            dist = np.linalg.norm(value - signature)
            if dist < min_dist:
                min_dist = dist
                identity = onlychar(key)
        # Check if the distance is above a certain threshold
        threshold = 1.0  # You can adjust this threshold based on your needs
        if min_dist > threshold:
            identity = 'Unknown'
            color = (0, 0, 255)  # Red for 'Unknown'
        else:
            color = (0, 255, 0)  # Green for known identities
        
        cv2.putText(gbr1, identity, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
        cv2.rectangle(gbr1, (x1, y1), (x2, y2), color, 2)
        if identity == 'Unknown':
            name_intruder = f"{uuid.uuid4()}.jpg"
            cv2.imwrite(os.path.join("intruders",name_intruder), gbr1)
            print(f"Unknown face captured and saved as {name_intruder}")
      
    cv2.imshow('res', gbr1)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cv2.destroyAllWindows()
cap.release()


1/1 [==============================] - 0s 53ms/step
Unknown face captured and saved as aad631a3-1012-4815-bbed-5aff5d1c3934.jpg
1/1 [==============================] - 0s 57ms/step
Unknown face captured and saved as aefd00b0-8833-4cc7-8c42-ab9f52acb0f0.jpg
1/1 [==============================] - 0s 56ms/step
Unknown face captured and saved as 656991c6-b8c6-46c5-a0e5-00dfba9ddc16.jpg
1/1 [==============================] - 0s 58ms/step
Unknown face captured and saved as 742f0500-c801-4cfa-9061-c5c5914bcab3.jpg
1/1 [==============================] - 0s 57ms/step
Unknown face captured and saved as f6e81a9f-22fd-4837-bc15-eff7a33ef5a7.jpg
1/1 [==============================] - 0s 57ms/step
Unknown face captured and saved as e4fb83b2-eb30-41fa-beae-48fc2c3dbaf7.jpg
1/1 [==============================] - 0s 59ms/step
Unknown face captured and saved as fda60c99-cc09-4b44-ae30-5c8762c4d41e.jpg
1/1 [==============================] - 0s 54ms/step
Unknown face captured and saved as edbaa6ae-cbde-4a7

In [6]:
# import os
# import re

# directory_path = "anchor/"  # Replace with the path to your pictures directory
# new_name = "Shepherd Umanah"

# # Get a list of all files with the .jpg extension in the directory
# jpg_files = [file for file in os.listdir(directory_path) if file.endswith(".jpg")]

# # Rename each file with your name and a number
# for i, old_name in enumerate(jpg_files, start=1):
#     new_name_with_number = f"{new_name}_{i}.jpg"
#     old_file_path = os.path.join(directory_path, old_name)
#     new_file_path = os.path.join(directory_path, new_name_with_number)
#     os.rename(old_file_path, new_file_path)